In [1]:
import pandas as pd
from pykrx import stock
from datetime import datetime, timedelta
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# 함수 정의

## 지정한 날짜가 영업일이 아닐 때 다음 영업일을 찾아 주는 함수

In [2]:
def get_next_trading_day(date):
    """
    입력 날짜가 휴장일이면 다음 영업일까지 반복해서 이동
    date: 'YYYYMMDD' 문자열
    """
    while True:
        # 삼성전자(005930)의 OHLCV 데이터 확인
        df = stock.get_market_ohlcv_by_date(date, date, "005930")
        if not df.empty:
            return date
        # 휴장일이면 다음날로 이동
        dt = datetime.strptime(date, "%Y%m%d") + timedelta(days=+1)
        print(f"{date}는 휴장일입니다. 다음 영업일을 검색합니다...")
        date = dt.strftime("%Y%m%d")

## 지정한 날짜가 영업일이 아닐 때 이전 영업일을 찾아 주는 함수

In [3]:
def get_previous_trading_day(date):
    """
    입력 날짜가 휴장일이면 다음 영업일까지 반복해서 이동
    date: 'YYYYMMDD' 문자열
    """
    while True:
        # 삼성전자(005930)의 OHLCV 데이터 확인
        df = stock.get_market_ohlcv_by_date(date, date, "005930")
        if not df.empty:
            return date
        # 휴장일이면 이전날로 이동
        dt = datetime.strptime(date, "%Y%m%d") - timedelta(days=+1)
        print(f"{date}는 휴장일입니다. 이전 영업일을 검색합니다...")
        date = dt.strftime("%Y%m%d")

## 지정한 기간동안의 코스피200의 종목 리스트를 반환하는 함수

In [4]:
def kospi200_list(start_year, end_year):
    # 결과를 저장할 리스트 생성
    records = []

    for year in range(start_year-1, end_year + 1):
        date = f"{year}0430"  # 4월말 기준 (휴장일일 경우 이후 영업입)
        trading_date = get_previous_trading_day(date)
        df = stock.get_index_portfolio_deposit_file("1028", trading_date, alternative = True)
        # 각 코드에 대해 연도와 함께 리스트에 저장
        for code in df:
            records.append({'연도': year, '주식코드': code})

    # 리스트를 데이터프레임으로 변환
    result_df = pd.DataFrame(records)

    frequency = result_df['주식코드'].value_counts()
    frequency_df = frequency.reset_index()
    frequency_df.columns = ['주식코드', '빈도수']
    code_df = frequency_df[['주식코드']]
    code_list = code_df['주식코드'].tolist()

    return code_list

# 기간 내 코스피 200 종목 리스트 수집

In [5]:
# 코스피200 종목 리스트 수집 
ticker_list = kospi200_list(2015, 2025)

# 누락된 2014년 코스피200 종목 데이터 수집 후 병합
list_df = pd.read_csv("data/2014_04_kospi200.csv",dtype={"종목코드" : str}, encoding='cp949')
list_df["날짜"] = "2014-04-30"
list_df["연도"] = "2014"
list_df = list_df[["종목코드","날짜","연도","상장시가총액"]]
list_df.rename(columns={'상장시가총액' : '시가총액', '종목코드' : '주식코드'}, inplace=True)
code_list = list_df["주식코드"].to_list()
ticker_list = ticker_list+code_list
ticker_list = list(set(ticker_list))

KRX web server does NOT provide data prior to 2014/05/01.
20160430는 휴장일입니다. 이전 영업일을 검색합니다...
20170430는 휴장일입니다. 이전 영업일을 검색합니다...
20170429는 휴장일입니다. 이전 영업일을 검색합니다...
20200430는 휴장일입니다. 이전 영업일을 검색합니다...
20220430는 휴장일입니다. 이전 영업일을 검색합니다...
20230430는 휴장일입니다. 이전 영업일을 검색합니다...
20230429는 휴장일입니다. 이전 영업일을 검색합니다...


# 수익률 수집

In [6]:
start_year = 2015
end_year = 2025

records = []
for year in range(start_year-1, end_year+1):
    start_date = f"{year}0430"
    trading_date = get_previous_trading_day(start_date)
    cap_df = stock.get_market_ohlcv_by_ticker(trading_date)
    for code in ticker_list:
        if code in cap_df.index:
            close = cap_df.loc[code, '종가']
            # open = cap_df.loc[code, '시가']
            records.append({
                '주식코드': code,
                '연도': year,
                '날짜': trading_date,
                '종가': close,
                # '시가': open
            })
        else:
            records.append({
                '주식코드': code,
                '연도': year,
                '날짜': trading_date,
                '종가': None,
                # '종가': None
            })

20160430는 휴장일입니다. 이전 영업일을 검색합니다...
20170430는 휴장일입니다. 이전 영업일을 검색합니다...
20170429는 휴장일입니다. 이전 영업일을 검색합니다...
20200430는 휴장일입니다. 이전 영업일을 검색합니다...
20220430는 휴장일입니다. 이전 영업일을 검색합니다...
20230430는 휴장일입니다. 이전 영업일을 검색합니다...
20230429는 휴장일입니다. 이전 영업일을 검색합니다...


In [12]:
df = pd.DataFrame(records)
df.sort_values(by=['주식코드','연도'],ascending=True,ignore_index=True,inplace=True)

df['수익률'] = df.groupby('주식코드')['종가'].pct_change()

df = df.replace([np.inf, -np.inf], np.nan)

df = df[df['수익률'] != -1]

df = df[['주식코드','연도','수익률']]
df.dropna(inplace=True)

In [13]:
df.to_excel("data/수익률.xlsx", index=False)

# 수익률과 재무지표와 합치기

In [14]:
industry_df = pd.read_excel("data/final_data.xlsx", dtype={'거래소코드' : str})

rtn_df = pd.read_excel("data/수익률.xlsx", dtype={'주식코드' : str})

In [15]:
def ts_data_loader(df):
    ts_list = df.columns.unique()

    ts_df = df.copy()
    ts_df['회계년도'] = ts_df['회계년도'].str.split('/').str[0].astype(int)
    ts_df['거래소코드'] = ts_df['거래소코드'].str.strip()
    ts_df.rename(columns = {'회계년도':'연도',
                            '거래소코드':'주식코드'}, inplace=True)
    ts_df = ts_df.dropna()

    # 중복 확인
    duplicates = ts_df.duplicated(subset=['주식코드', '연도'], keep=False)
    ts_df[duplicates]

    # 중복된 ('주식코드', '연도') 조합에서 첫 번째 값만 남김
    ts_df = ts_df.drop_duplicates(subset=['주식코드', '연도'], keep='first').sort_values(by=['주식코드','연도'])

    return ts_df

In [16]:
industry_data = ts_data_loader(industry_df)

In [17]:
final_industry_data = pd.merge(industry_data, rtn_df, on=['주식코드','연도'], how='inner')
final_industry_data_rtn = final_industry_data.replace([np.inf, -np.inf], np.nan)

In [18]:
# (원)단위 컬럼 제거
col_list = []
for col in final_industry_data_rtn.columns:
    if '원)' in col:
        col_list.append(col)
# print(col_list)
final_industry_data_rtn.drop(columns=col_list,inplace=True)

In [ ]:
final_industry_data_rtn = final_industry_data_rtn.drop(columns = ["회사명","주식코드","연도"])

,총자본증가율(IFRS),유형자산증가율(IFRS),비유동생물자산증가율(IFRS),투자부동산증가율(IFRS),비유동자산증가율(IFRS),유동자산증가율(IFRS),재고자산증가율(IFRS),자기자본증가율(IFRS),매출액증가율(IFRS),정상영업이익증가율(IFRS),...,기계장비율(IFRS),자본집약도(IFRS),총자본투자효율(IFRS),설비투자효율(IFRS),기계투자효율(IFRS),부가가치율(IFRS),노동소득분배율(IFRS),자본분배율(IFRS),이윤분배율(IFRS),수익률
0,-1.22,-4.16,0.0,-1.29,-1.31,0.11,8.61,3.10,19.75,20.36,...,46.29,2047.99,8.57,41.80,379.17,35.08,19.66,80.34,20.01,0.962547
1,-1.79,-3.70,0.0,-1.30,-1.64,-3.85,-10.48,2.75,1.99,10.22,...,0.00,0.00,8.89,44.22,502.67,35.04,19.01,80.99,28.16,-0.335878
2,-3.07,-7.75,0.0,-1.26,-2.50,-11.26,-33.22,3.00,-8.21,-7.44,...,0.00,0.00,9.12,47.66,647.91,37.96,18.25,81.75,23.48,-0.915517
3,2.29,-6.58,0.0,-1.13,0.96,23.35,58.28,2.56,-3.13,-1.11,...,0.00,0.00,7.22,41.34,1552.83,31.75,19.89,80.11,32.96,0.013605
4,5.68,-69.74,0.0,5.71,-5.79,153.86,-60.76,1.58,-9.15,-0.01,...,0.00,0.00,6.22,134.74,18933.42,31.82,15.85,84.15,27.55,-0.248322
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2597,-5.28,0.67,0.0,0.00,-2.11,-9.02,4.27,-3.89,12.40,15.41,...,0.00,0.00,41.61,150.64,66956.19,9.49,41.78,58.22,20.13,-0.069881
2598,32.54,82.49,0.0,0.00,34.26,29.85,9.24,38.51,4.83,0.65,...,0.00,0.00,36.91,907.64,0.00,38.39,8.78,91.22,63.87,0.000000
2599,9.19,297.21,0.0,0.00,36.05,-34.33,-11.57,17.77,-10.37,-23.65,...,0.00,0.00,28.54,140.78,0.00,36.16,11.37,88.63,58.94,-0.532624
2600,-1.77,-1.78,0.0,0.00,-5.59,111.96,0.00,-7.60,-69.96,-81.79,...,5.57,77491.44,-2.38,-694.84,-33101.32,-98.60,0.00,0.00,0.00,-0.194497


In [21]:
final_industry_data_rtn.to_excel("data/제조업_수익률.xlsx", index=False)